# **Alcohol, car crashes and bars in Allegheny County,PA**

## Alcohol Use in the United States:
**Prevalence of Drinking:** According to the 2015 National Survey on Drug Use and Health (NSDUH), 86.4 percent of people ages 18 or older reported that they drank alcohol at some point in their lifetime; 70.1 percent reported that they drank in the past year; 56.0 percent reported that they drank in the past month.

**Prevalence of Binge Drinking and Heavy Alcohol Use:** In 2015, 26.9 percent of people ages 18 or older reported that they engaged in binge drinking in the past month; 7.0 percent reported that they engaged in heavy alcohol use in the past month.2 (See "Definitions" box for definitions of binge drinking and heavy alcohol use.)
### Alcohol-Related Deaths:
An estimated **88,0008 people** (approximately 62,000 men and 26,000 women8) die from alcohol-related causes annually, making alcohol the **third leading preventable cause of death in the United States**. The first is tobacco, and the second is poor diet and physical inactivity.
Only in 2014, alcohol-impaired driving fatalities accounted for **9,967 deaths** (31 percent of overall driving fatalities).

<a href="https://www.niaaa.nih.gov/alcohol-health/overview-alcohol-consumption/alcohol-facts-and-statistics">Source</a>


## Car Crashes in the United States:

The National Safety Council estimates that motor vehicle deaths claimed 40,100 lives in 2017 versus the 2016 total of 40,327. The 2017 assessment is 6% higher than the number of deaths in 2015. If the estimate holds, it will be the second consecutive year that motor vehicle deaths topped 40,000.

About 4.57 million people were injured seriously enough to require medical attention in motor vehicle crashes in 2017, and costs to society totaled $413.8 billion. Both figures are about 1% lower than 2016 calculations.

<a href="https://www.nsc.org/road-safety/safety-topics/fatality-estimates">Source</a>


## Introduction

In this study we will verify if there is any relationship between car crashes and the proximity bars/pubs to where the accident happened in Allegheny County in Pennsylvania (a radius of 5 miles will be used to find if there's any venue that sells alcohol around) using datasets provided by the <a href="https://www.data.gov/catalog-help/">Data.gov</a>, and the venue search tool provided by the <a href="https://developer.foursquare.com/">Foursquare API</a>. 

First, lets import a few libraries that are going to be used:

In [1]:
import pandas as pd
import numpy as np

Now, lets load the data into a Pandas data frame the data containing Car Crashes in Allegheny County. This data set can be downloaded <a href="https://data.wprdc.org/datastore/dump/2c13021f-74a9-4289-a1e5-fe0472c89881">here</a> and contain data about car crashes from 2004 until 2017.

In [2]:
car_crashes = pd.read_csv("https://data.wprdc.org/datastore/dump/2c13021f-74a9-4289-a1e5-fe0472c89881")
print("Data set loaded")

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (79,183) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data set loaded


Lets check the size of the data frame:

In [3]:
print("The data set car crashes has", car_crashes.shape)

The data set car crashes has (170365, 190)


As we can see the data set has **170.365 entries** and **190 columns**.

Here is a quick peek of what it looks like:

In [4]:
car_crashes.head()

,CRASH_CRN,DISTRICT,CRASH_COUNTY,MUNICIPALITY,POLICE_AGCY,CRASH_YEAR,CRASH_MONTH,DAY_OF_WEEK,TIME_OF_DAY,HOUR_OF_DAY,...,LANE_COUNT,RDWY_ORIENT,ROAD_OWNER,ROUTE,SPEED_LIMIT,SEGMENT,OFFSET,STREET_NAME,TOT_INJ_COUNT,SCHOOL_BUS_UNIT
0,2004000001,11,2,2454,02454,2004,1,5,257.0,2.0,...,2.0,E,2.0,2032,35.0,10.0,1000.0,LEWIS RUN RD,NaN,NaN
1,2004000010,11,2,2440,02440,2004,1,5,1027.0,10.0,...,2.0,W,3.0,NaN,35.0,NaN,NaN,COAL VALLEY RD,NaN,NaN
2,2004000016,11,2,2203,02203,2004,1,5,1839.0,18.0,...,2.0,W,2.0,3082,35.0,NaN,NaN,CLINTON FRANKFORT RD,NaN,NaN
3,2004000022,11,2,2404,02404,2004,1,6,914.0,9.0,...,2.0,N,4.0,NaN,25.0,NaN,NaN,NORTH BALPH AV,NaN,NaN
4,2004000030,11,2,2419,02419,2004,1,5,2230.0,22.0,...,2.0,S,4.0,NaN,20.0,NaN,NaN,THORNBERRY DR,NaN,NaN


For this study many of the columns listed won't be used. So, to make things a little easier to understand, lets clean up our data set by dropping the data that won't be used.

In [5]:
car_crashes_c = car_crashes[['PERSON_COUNT', 'VEHICLE_COUNT', 'FATAL_COUNT', 'INJURY_COUNT', 'DEC_LAT', 'DEC_LONG', 'DRINKING_DRIVER', 'STREET_NAME']]

In [6]:
car_crashes_c.head()

,PERSON_COUNT,VEHICLE_COUNT,FATAL_COUNT,INJURY_COUNT,DEC_LAT,DEC_LONG,DRINKING_DRIVER,STREET_NAME
0,1,1,0,0,40.3234,-79.9404,1,LEWIS RUN RD
1,1,1,0,1,40.3182,-79.9076,0,COAL VALLEY RD
2,1,1,0,1,40.4886,-80.3371,0,CLINTON FRANKFORT RD
3,3,2,0,1,NaN,NaN,0,NORTH BALPH AV
4,1,2,0,0,NaN,NaN,0,THORNBERRY DR


In [7]:
car_crashes_c.shape

(170365, 8)

OK, that's a little better! From the example above we can see that there still are two things that need to be taken care of before we start. 

- We need to filter all the accidents that do not involve a drunk driver.
- There are a some entries that do not have the latitude and longitude.

We will start by filtering out all non-drinking related accidents in order to have fewer entries that we need to address the lack of coodinantes.

In [8]:

car_crashes_c = car_crashes_c.replace(0, pd.np.nan).dropna(axis=0, how='any', subset=['DRINKING_DRIVER']).reset_index(drop=True)
car_crashes_c.shape


(15316, 8)

As we can see, by leaving only the accidents in which the driver had ingested alcohol we have now a total of **15.316 entries**.

In [9]:
car_crashes_c.head()


,PERSON_COUNT,VEHICLE_COUNT,FATAL_COUNT,INJURY_COUNT,DEC_LAT,DEC_LONG,DRINKING_DRIVER,STREET_NAME
0,1.0,1.0,NaN,NaN,40.3234,-79.9404,1.0,LEWIS RUN RD
1,2.0,1.0,NaN,1.0,40.3320,-79.9542,1.0,NATIONAL DR
2,2.0,2.0,NaN,NaN,40.5741,-80.1582,1.0,FERN HOLLOW RD
3,1.0,1.0,NaN,1.0,40.5234,-80.1771,1.0,FOURTH AV
4,5.0,2.0,NaN,NaN,NaN,NaN,1.0,SHARPSHILL RD


Now, lets try to fix the absence of coordinates of some of the entries.

First by dropping the cases in which we don't have neither the street name nor the coordinates, because in that case we won't be able to retrieve any positional information.

In [10]:
for index, row in car_crashes_c.iterrows():
    if (np.isnan(row['DEC_LAT']) and pd.isnull(row['STREET_NAME'])):
        car_crashes_c.drop(car_crashes_c.index[index], inplace=True)
car_crashes_c.shape

(15282, 8)

We proceed by obtaining the coordinates for the entries that only have the street address:

In [11]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00   1.05 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   1.61 MB/s


Lets verify how many entries don't have coordinates:

In [12]:
i=0
for index, row in car_crashes_c.iterrows():
    if np.isnan(row['DEC_LAT']):
        i+=1
print("total", i)

total 1013


Now, lets try to fix the absence of coordinates of some of the entries. (this operation takes a while to complete due the use of the geolocator API)

In [25]:
for index, row in car_crashes_c.iterrows():
    if np.isnan(row['DEC_LAT']):
        address = str(row['STREET_NAME']) + ', PA'
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        if (pd.isnull(location)):
            car_crashes_c.drop(index, inplace=True)
        else: 
            car_crashes_c.loc[index, 'DEC_LAT']=location.latitude
            car_crashes_c.loc[index, 'DEC_LONG']=location.longitude
    
print("Done retrieving the coordinates!")

Done retrieving the coordinates!


Alright! Time to check how many entries we have left. Note that the addresses that geolocator couldn't find were dropped.

In [32]:
car_crashes_c.reset_index(drop=True, inplace=True)
car_crashes_c.shape

(14721, 8)

Our data frame now looks like this:

In [33]:
car_crashes_c[0:50]

,PERSON_COUNT,VEHICLE_COUNT,FATAL_COUNT,INJURY_COUNT,DEC_LAT,DEC_LONG,DRINKING_DRIVER,STREET_NAME
0,1.0,1.0,NaN,NaN,40.323400,-79.940400,1.0,LEWIS RUN RD
1,2.0,1.0,NaN,1.0,40.332000,-79.954200,1.0,NATIONAL DR
2,2.0,2.0,NaN,NaN,40.574100,-80.158200,1.0,FERN HOLLOW RD
3,1.0,1.0,NaN,1.0,40.523400,-80.177100,1.0,FOURTH AV
4,5.0,2.0,NaN,NaN,40.507039,-79.936459,1.0,SHARPSHILL RD
5,1.0,1.0,NaN,1.0,40.340100,-79.967300,1.0,CURRY HOLLOW RD
6,1.0,1.0,NaN,NaN,40.324926,-79.985527,1.0,COCHRAN MILL RD
7,2.0,1.0,NaN,1.0,40.498100,-80.067700,1.0,CHESTNUT ST
8,2.0,2.0,NaN,2.0,40.667200,-80.101600,1.0,BRUSH CREEK RD
9,1.0,1.0,NaN,1.0,40.680538,-74.178951,1.0,ELIZABETH AVENUE AV
